In [1]:
from matplotlib import pyplot as plt
import importlib
import logging

from collections import Counter

import numpy as np
import pandas as pd

from src.data_import import Importing as importing
from src.fqi.FQI import FQI
from src.policy.QPolicy import QPolicy
from src.rewards.Position import Position
from src.rewards.UnrealizedReward import UnrealizedReward
from src.utils import set_position, create_all_combination, is_parallelizable

In [2]:
def random_action(state):
    the_random_action = pd.Series(np.random.randint(-1, 2, state.shape[0]), dtype='category') #values: -1, 0, 1
    return the_random_action.rename('action')
def possible_actions(integer_positions):
    return [Position(integer) for integer in integer_positions]

In [3]:
start_date = "01/01"
end_date = "12/31"
historical_observations =2
columns_with_historical = [ "log_close"      ]
starting_time = 60
columns_with_current = ["open", "close"]
fee =1e-05
position_size = 10000
action_set = [1,-1,0]
model_name = "RandomForest"
model_parameters= {
            "n_estimators":4 ,
            "criterion": "mse",
            "n_jobs": -1,
            "min_impurity_decrease": 0.0001
        }
max_iteration =5
discount =0.99
run_parameters = {}

In [4]:
print('Reading training set from file  ...')
file_data = importing.import_df("2016-EURUSD-1m.csv")
print('...Done')




Reading training set from file  ...
...Done


In [5]:
file_data

,date,time_min,close,open,high,low,minute,log_open,log_close,log_high,log_low,volume,datetime
0,01/04,01:00,1.0847,1.0846,1.0847,1.0845,60.0,0.000000,0.000092,0.000092,-0.000092,1.000000,2016-01-04 01:00:00
1,01/04,01:01,1.0846,1.0847,1.0847,1.0846,61.0,0.000092,0.000000,0.000092,0.000000,1.038156,2016-01-04 01:01:00
2,01/04,01:02,1.0845,1.0846,1.0846,1.0845,62.0,0.000000,-0.000092,0.000000,-0.000092,1.322734,2016-01-04 01:02:00
3,01/04,01:03,1.0843,1.0845,1.0845,1.0842,63.0,-0.000092,-0.000277,-0.000092,-0.000369,1.701113,2016-01-04 01:03:00
4,01/04,01:04,1.0843,1.0843,1.0843,1.0843,64.0,-0.000277,-0.000277,-0.000277,-0.000277,1.915739,2016-01-04 01:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
319772,12/30,21:25,1.0518,1.0520,1.0520,1.0518,1285.0,-0.003037,-0.003227,-0.003037,-0.003227,0.227612,2016-12-30 21:25:00
319773,12/30,21:26,1.0518,1.0518,1.0518,1.0518,1286.0,-0.003227,-0.003227,-0.003227,-0.003227,0.199005,2016-12-30 21:26:00
319774,12/30,21:27,1.0520,1.0518,1.0520,1.0518,1287.0,-0.003227,-0.003037,-0.003037,-0.003227,0.269071,2016-12-30 21:27:00
319775,12/30,21:28,1.0520,1.0520,1.0520,1.0519,1288.0,-0.003037,-0.003037,-0.003037,-0.003132,0.225124,2016-12-30 21:28:00


In [6]:
# Volatility Indicator Function: ATR - Average True Range
talib_fun = [ {"name":"ATR", "parameters": {"timeperiod": 10}}]
print('Applying talib functions to training set: {talib} ...'.format(talib=talib_fun))
file_data_talib, talib_names = importing.apply_talib(file_data,    talib_fun )
print('...Done')


Applying talib functions to training set: [{'name': 'ATR', 'parameters': {'timeperiod': 10}}] ...
...Done


In [7]:
print('Creating training dataframes for dates {start} to {end} ...'.format(start=start_date,end=end_date))
current_state_no_position, next_state_no_position, price_info, minutes =  importing.create_tuples(  start_date,end_date,file_data_talib, historical_observations,columns_with_historical, columns_with_current+ talib_names, starting_time)
print('...Done')

Creating training dataframes for dates 01/01 to 12/31 ...
...Done


In [8]:
current_state_no_position


,log_close_-2,log_close_-1,log_close_0,open,close,ATR
0,0.001382,0.001290,0.001750,1.0860,1.0865,0.000271
1,0.001290,0.001750,0.001842,1.0865,1.0866,0.000294
2,0.001750,0.001842,0.001658,1.0866,1.0864,0.000295
3,0.001842,0.001658,0.001750,1.0864,1.0865,0.000295
4,0.001658,0.001750,0.001474,1.0865,1.0862,0.000296
...,...,...,...,...,...,...
303912,-0.003513,-0.003513,-0.003037,1.0515,1.0520,0.000143
303913,-0.003513,-0.003037,-0.003227,1.0520,1.0518,0.000149
303914,-0.003037,-0.003227,-0.003227,1.0518,1.0518,0.000134
303915,-0.003227,-0.003227,-0.003037,1.0518,1.0520,0.000141


In [9]:
next_state_no_position #1 day after wrt current_state_no_position

,log_close_-2,log_close_-1,log_close_0,open,close,ATR
0,0.001290,0.001750,0.001842,1.0865,1.0866,0.000294
1,0.001750,0.001842,0.001658,1.0866,1.0864,0.000295
2,0.001842,0.001658,0.001750,1.0864,1.0865,0.000295
3,0.001658,0.001750,0.001474,1.0865,1.0862,0.000296
4,0.001750,0.001474,0.001842,1.0862,1.0866,0.000316
...,...,...,...,...,...,...
303912,-0.003513,-0.003037,-0.003227,1.0520,1.0518,0.000149
303913,-0.003037,-0.003227,-0.003227,1.0518,1.0518,0.000134
303914,-0.003227,-0.003227,-0.003037,1.0518,1.0520,0.000141
303915,-0.003227,-0.003037,-0.003037,1.0520,1.0520,0.000137


In [10]:
action = random_action(current_state_no_position)
current_state, next_state = set_position(current_state_no_position, next_state_no_position, action, minutes)

    # reward
reward = UnrealizedReward(fee, position_size).calculate(current_state, action, price_info, minutes)

In [11]:
action

0         1
1         0
2         1
3        -1
4        -1
         ..
303912    1
303913    1
303914    0
303915    0
303916    0
Name: action, Length: 303917, dtype: category
Categories (3, int64): [-1, 0, 1]

In [12]:
reward

0        -0.100000
1         0.820302
2        -0.100000
3         0.720387
4         2.761922
            ...   
303912    4.752852
303913   -1.901502
303914   -0.100000
303915    0.000000
303916    0.000000
Name: position, Length: 303917, dtype: float64

In [13]:
    # samples creation
samples = { 'current_state': current_state.copy(),'next_state': next_state.copy(), 'reward': reward, 'action': action,  'minute': minutes,  'fee': fee, 'position_size': position_size,    'price_info': price_info }
samples = create_all_combination(samples, possible_actions(action_set))
print('Initializing model {model} with parameters {parameters} ...' .format(model=model_name,  parameters=model_parameters))
model_module = importlib.import_module( '.' + model_name,'src.models')\
    
model = model_module.get_model(model_parameters, samples['current_state'].copy(),   samples['reward'], samples['action'])
print('...Done')

Initializing model RandomForest with parameters {'n_estimators': 4, 'criterion': 'mse', 'n_jobs': -1, 'min_impurity_decrease': 0.0001} ...
...Done


In [14]:
samples

{'current_state':         log_close_-2  log_close_-1  log_close_0    open   close       ATR  \
 0           0.001382      0.001290     0.001750  1.0860  1.0865  0.000271   
 1           0.001290      0.001750     0.001842  1.0865  1.0866  0.000294   
 2           0.001750      0.001842     0.001658  1.0866  1.0864  0.000295   
 3           0.001842      0.001658     0.001750  1.0864  1.0865  0.000295   
 4           0.001658      0.001750     0.001474  1.0865  1.0862  0.000296   
 ...              ...           ...          ...     ...     ...       ...   
 303912     -0.003513     -0.003513    -0.003037  1.0515  1.0520  0.000143   
 303913     -0.003513     -0.003037    -0.003227  1.0520  1.0518  0.000149   
 303914     -0.003037     -0.003227    -0.003227  1.0518  1.0518  0.000134   
 303915     -0.003227     -0.003227    -0.003037  1.0518  1.0520  0.000141   
 303916     -0.003227     -0.003037    -0.003037  1.0520  1.0520  0.000137   
 
         position  
 0              1  
 1   

In [ ]:
logger = logging.getLogger("Calibration")
fqi_configuration = {
        'possible_actions': possible_actions(action_set),
        'max_iterations': max_iteration,
        'discount': discount,
        'sample_iterations': 1 
}
print('Running FQI with parameters {parameters} ...'.format(parameters=fqi_configuration))
if is_parallelizable(model):
        model.set_params(n_jobs=-1)
fqi = FQI(samples, model, fqi_configuration, logger)
fitted_model, q_norms, losses = fqi.run(**(run_parameters))
print('...Done')


Running FQI with parameters {'possible_actions': [<Position.L: 1>, <Position.S: -1>, <Position.F: 0>], 'max_iterations': 5, 'discount': 0.99, 'sample_iterations': 1} ...


In [ ]:
  # optimal policy applied to training
print('Applying optimal policy to training set...')
current_state_train = current_state_no_position.copy()
current_state_train['position'] = Position.F
if is_parallelizable(model):
    fitted_model.set_params(n_jobs=1)
policy = QPolicy(fitted_model)
optimal_state_train, optimal_actions_train = policy.apply(minutes, current_state_train, possible_actions(action_set))
reward_train = UnrealizedReward(fee,position_size).calculate(optimal_state_train,  optimal_actions_train,  price_info, minutes)
print('...Done')


In [ ]:
print('Reading testing set from file  ...')
file_data_testing  = importing.import_df("2017-EURUSD-1m.csv")
print('...Done')




In [ ]:
print('Applying talib functions to testing set: {talib} ...'.format(talib=talib_fun))
file_data_testing_talib, talib_names  = importing.apply_talib(file_data_testing,    talib_fun )
print('...Done')


print('Creating training dataframes for dates {start} to {end} ...'.format(start=start_date,end=end_date))
current_state_testing,  next_state_testing, price_info_testing,  minutes_testing  =  importing.create_tuples(start_date, end_date, file_data_testing_talib, historical_observations,columns_with_historical, columns_with_current+ talib_names, starting_time)

print('...Done')

In [ ]:
print('Applying optimal policy to testing set...')
current_state_testing['position'] = Position.F
if is_parallelizable(model):
    fitted_model.set_params(n_jobs=1)
policy = QPolicy(fitted_model)
optimal_state, optimal_actions = policy.apply(minutes_testing,current_state_testing,possible_actions(action_set))
print('...Done')

print('Calculating optimal reward achieved in testing set...')
reward_testing = UnrealizedReward(fee, position_size).calculate(optimal_state,optimal_actions, price_info_testing, minutes_testing)
print('...Done')

In [ ]:
print('############################  Output  ############################')
print('Sum of optimal Reward(training set): {reward}'.format(reward=sum(reward_train)))
print('Sum of optimal Reward(testing set): {reward}'.format(reward=sum(reward_testing)))
print('Optimal action summary (training set): {summary}'.format(summary=Counter(optimal_actions_train)))
print('Optimal action summary (testing set): {summary}'.format(summary=Counter(optimal_actions)))

In [ ]:
plt.plot(np.cumsum(reward_train))
plt.show()
plt.plot(np.cumsum(reward_testing))
plt.show()